In [1]:
from obspy import read_inventory, UTCDateTime
from obspy.core.event import read_events
from obspy import read
import os
import subprocess

%matplotlib inline

"""
Double check that obspy is actually using an butterworth filter
Plot out some of the spectra for this data
"""

'\nDouble check that obspy is actually using an butterworth filter\nPlot out some of the spectra for this data\n'

In [2]:
path=os.getcwd()
EVID=385392
comparisionFrequency = 3.3
plot = False #plot each response correction?
def write_st_as_sac(st,instrument="acc",eventID=0):
    directory = './'+str(eventID)+"/response"
    if not os.path.exists(directory):
        os.makedirs(directory)
    directory = directory + "/" + instrument + "/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    fname = directory+"/%s.sac" % trace.id
    trace.write(fname, format="SAC")

In [3]:
### Clean up the input data
command,newFile = '',''

try:
    os.mkdir(path+"/"+str(EVID)+"/"+"cleaned")
    inputFiles = os.listdir(path+"/"+str(EVID))
    for file in inputFiles:
        net = "NN"
        sta,cha = file.split(".")[0],file.split(".")[1]
        newFile= net+'.'+sta+".."+cha
        command = "cp " + path+'/'+str(EVID)+'/'+file + ' ' + path+'/'+str(EVID)+'/'+"cleaned"+"/"+newFile + ".sac"
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        
except Exception as e:
    print(repr(e))
    print("you have most likely already cleaned up the input files")

FileExistsError(17, 'File exists')
you have most likely already cleaned up the input files


In [4]:
#begin by loading the station xml

#"/home/dukeleto/git/ssaModels/actualData/sacThomasCreekIris/backupStationXML.xml"
inventory = read_inventory(path+"/reno.xml")

In [5]:
#get all of the availible stations
st = read(path+'/'+str(EVID)+"/cleaned/*.sac")
#fix them so that they have the correct network metadata 
#(so that they can be found in the instrument response)
for i in st:
    i.meta["network"] = "NN"

In [6]:
#Play with some filters and look at how well it works
"""
r1 = st[190].copy()
r2 = st[191].copy()
#filter down to target frequencty
r1Copy = r1.copy()
r1.filter('lowpass',freq=comparisionFrequency,corners=5,zerophase=True)
r1.plot()
r1Copy.plot()
r1.remove_response(inventory=inventory,output="acc",plot=True)
"""
def filterACC(trace,corners=True,zerophase=True,comparisionFrequency=comparisionFrequency):
    #detrend
    trace.detrend('simple')
    trace.filter('lowpass',freq=comparisionFrequency,corners=5,zerophase=True)
    pass

In [7]:
#remove the response for all of the stations
#pre_filt = (0.05, 0.1, 3.3, 3.5)
for trace in st:
    if(trace.meta["sac"]["cmpaz"] != 0):
        print("WARN: station " + str(trace) + "has not been rotated!")
    
    #attempt to remove the response for each
    try:
        print("correcting " + str(trace))
        #determine if accelerogram--IACC=08 (idep= independent variable)
        if(trace.meta['sac']['idep']==8):
            filterACC(trace)
            trace.remove_response(inventory=inventory,output="acc",plot=plot)
            #save to accelerogram dir
            write_st_as_sac(trace,instrument="acc",eventID=EVID)
            
        #velocity--IVEL=7
        elif(trace.meta['sac']['idep']==7):
            filterACC(trace)
            trace.remove_response(inventory=inventory,output="vel",plot=plot)            
            write_st_as_sac(trace,instrument="vel",eventID=EVID)
        #displacement
        elif(trace.meta['sac']['idep']==6):
            filterACC(trace)
            trace.remove_response(inventory=inventory,output="disp",plot=plot)
            write_st_as_sac(trace,instrument="disp",eventID=EVID)
            
        else:
            print("unkown units for " + str(trace))
            print(trace.meta['sac']['idep'])
        #print("corrected response for " + str(trace))
    except Exception as e:
        #print("problem with trace")
        #print(str(trace))
        #print(repr(e))
        pass

correcting NN.109C..BHE | 2008-04-26T06:40:10.470000Z - 2008-04-26T06:41:10.445000Z | 40.0 Hz, 2400 samples
correcting NN.116A..BHE | 2008-04-26T06:40:10.475000Z - 2008-04-26T06:41:10.450000Z | 40.0 Hz, 2400 samples
correcting NN.116A..BHN | 2008-04-26T06:40:10.475000Z - 2008-04-26T06:41:10.450000Z | 40.0 Hz, 2400 samples
correcting NN.ADH..SHZ | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.435874Z | 50.0 Hz, 3000 samples
correcting NN.AL5..EHE | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.AL5..EHN | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.AL5..HGE | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.AL5..HGN | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.AL5S..EHE | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.AL5S..HGN | 20

/data/home/eeckert/anaconda3/envs/obspy_dev/lib/python3.6/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):


correcting NN.BEK..BHZ | 2008-04-26T06:40:10.470000Z - 2008-04-26T06:41:10.445000Z | 40.0 Hz, 2400 samples
correcting NN.BEN..SHZ | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.435874Z | 50.0 Hz, 3000 samples
correcting NN.BFC..SHZ | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.435874Z | 50.0 Hz, 3000 samples
correcting NN.BHP..SHZ | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.435874Z | 50.0 Hz, 3000 samples
correcting NN.BMHS..HHN | 2008-04-26T06:40:10.460000Z - 2008-04-26T06:41:10.450000Z | 100.0 Hz, 6000 samples
correcting NN.BMHS..HHZ | 2008-04-26T06:40:10.460000Z - 2008-04-26T06:41:10.450000Z | 100.0 Hz, 6000 samples
correcting NN.BMN..BHE | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.435874Z | 50.0 Hz, 3000 samples
correcting NN.BMN..BHN | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.435874Z | 50.0 Hz, 3000 samples
correcting NN.BMR..SHZ | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.435874Z | 50.0 Hz, 3000 samples
correcting NN.BON..SHZ | 2008-04-

/data/home/eeckert/anaconda3/envs/obspy_dev/lib/python3.6/site-packages/obspy/signal/filter.py:151: UserWarning: Selected corner frequency is above Nyquist. Setting Nyquist as high corner.
  warnings.warn(msg)


correcting NN.WHR..BHZ | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.435874Z | 50.0 Hz, 3000 samples
correcting NN.WIL..EHE | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.WIL..EHN | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.WIL..EHZ | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.WLAM..BHE | 2008-04-26T06:40:10.465000Z - 2008-04-26T06:41:10.440000Z | 40.0 Hz, 2400 samples
correcting NN.WLAM..BHN | 2008-04-26T06:40:10.465000Z - 2008-04-26T06:41:10.440000Z | 40.0 Hz, 2400 samples
correcting NN.WLAM..BHZ | 2008-04-26T06:40:10.465000Z - 2008-04-26T06:41:10.440000Z | 40.0 Hz, 2400 samples
correcting NN.WLD..EHN | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.WLD..EHZ | 2008-04-26T06:40:10.455000Z - 2008-04-26T06:41:10.445000Z | 100.0 Hz, 6000 samples
correcting NN.WLD..HGE | 2008

In [8]:
print("you should move all of these files up one directory for the convenience of youre post process program")

you should move all of these files up one directory for the convenience of youre post process program
